In [ ]:
import requests
import time
import pandas as pd
import os
import json

# ========================
# Funciones auxiliares
# ========================

def obtener_info_juego_completa(appid):
    """Consulta la API de Steam y extrae campos útiles para recomendación + interfaz."""
    url = f"https://store.steampowered.com/api/appdetails?appids={appid}"
    try:
        r = requests.get(url, timeout=10)
        data = r.json()
        juego_data = data.get(str(appid), {})
        if juego_data.get("success"):
            info = juego_data["data"]
            return {
                "appid": appid,
                "name": info.get("name"),
                "genres": [g["description"] for g in info.get("genres", [])],
                "developers": info.get("developers", []),
                "short_description": info.get("short_description", ""),
                "header_image": info.get("header_image", ""),
                "categories": [c["description"] for c in info.get("categories", [])],
                "release_date": info.get("release_date", {}).get("date", ""),
                "metacritic_score": info.get("metacritic", {}).get("score", None),
                "price": info.get("price_overview", {}).get("final_formatted", None),
            }
        else:
            print(f"❌ Juego {appid} no disponible en Steam (success: false).")
    except Exception as e:
        print(f"⚠️ Error con appid {appid}: {e}")
    return None


def guardar_resultados(resultados, archivo):
    if os.path.exists(archivo):
        existentes = pd.read_csv(archivo)
        nuevos = pd.DataFrame(resultados)
        combinados = pd.concat([existentes, nuevos], ignore_index=True)
        combinados = combinados.drop_duplicates(subset="appid")
    else:
        combinados = pd.DataFrame(resultados)
    combinados.to_csv(archivo, index=False)


def guardar_json_sin_duplicados(lista, archivo):
    existentes = []
    if os.path.exists(archivo):
        with open(archivo) as f:
            existentes = json.load(f)
    actualizados = list(set(existentes).union(set(lista)))
    with open(archivo, "w") as f:
        json.dump(actualizados, f)


def cargar_json(archivo):
    if os.path.exists(archivo):
        with open(archivo) as f:
            return json.load(f)
    return []

def cargar_ya_procesados(archivo):
    if os.path.exists(archivo):
        try:
            return pd.read_csv(archivo)["appid"].astype(str).tolist()
        except:
            return []
    return []

# ========================
# Proceso principal en bucle
# ========================

def ejecutar_descarga_completa(lista_ids, archivo_salida="steam_juegos_metadata.csv",
                                archivo_pendientes="pendientes_steam.json", max_intentos=5):

    procesados = set(cargar_ya_procesados(archivo_salida))
    pendientes = set(cargar_json(archivo_pendientes))
    objetivo = set(lista_ids)

    # Intentar hasta agotar intentos
    for intento in range(1, max_intentos + 1):
        print(f"\n🔁 RONDA {intento}/{max_intentos}")
        ids_a_hacer = list((objetivo - procesados) | pendientes)
        if not ids_a_hacer:
            print("✅ Todos los juegos han sido procesados correctamente.")
            break

        print(f"🎯 Juegos restantes: {len(ids_a_hacer)}")
        resultados = []
        fallidos = []

        for i, appid in enumerate(ids_a_hacer, 1):
            print(f"⏳ [{i}/{len(ids_a_hacer)}] Consultando appid: {appid}")
            info = obtener_info_juego_completa(appid)
            if info:
                resultados.append(info)
                procesados.add(str(appid))
            else:
                fallidos.append(str(appid))

            if i % 10 == 0 or i == len(ids_a_hacer):
                guardar_resultados(resultados, archivo_salida)
                guardar_json_sin_duplicados(fallidos, archivo_pendientes)
                resultados = []

            time.sleep(0.2)  # evitar rate limit

    print("🏁 Proceso finalizado tras completar todas las rondas.")


In [2]:
import pandas as pd

# --------------------------
# Cargar item_id desde juegos.csv
# --------------------------
def cargar_ids_desde_csv(ruta_csv="juegos.csv"):
    df = pd.read_csv(ruta_csv)
    df["item_id"] = df["item_id"].astype(str)
    return df["item_id"].tolist()

# --------------------------
# Ejecutar proceso principal
# --------------------------
from pathlib import Path

# Asegura que funciones auxiliares están definidas
# (Ya las definimos en tu script anterior: obtener_info_juego_completa, guardar_resultados, etc.)

# Cargar item_ids desde juegos.csv
lista_ids = cargar_ids_desde_csv("juegos.csv")

# Ejecutar scraping completo con reintentos y guardado incremental
ejecutar_descarga_completa(
    lista_ids,
    archivo_salida="steam_juegos_metadata.csv",
    archivo_pendientes="pendientes_steam.json",
    max_intentos=5
)



🔁 RONDA 1/5
🎯 Juegos restantes: 10978
⏳ [1/10978] Consultando appid: 384750
⏳ [2/10978] Consultando appid: 512360
⏳ [3/10978] Consultando appid: 340340
⏳ [4/10978] Consultando appid: 473460
⏳ [5/10978] Consultando appid: 10630
❌ Juego 10630 no disponible en Steam (success: false).
⏳ [6/10978] Consultando appid: 397670
⏳ [7/10978] Consultando appid: 357200
⏳ [8/10978] Consultando appid: 35470
❌ Juego 35470 no disponible en Steam (success: false).
⏳ [9/10978] Consultando appid: 458300
⏳ [10/10978] Consultando appid: 13510
⏳ [11/10978] Consultando appid: 329380
⏳ [12/10978] Consultando appid: 348470
⏳ [13/10978] Consultando appid: 410110
⏳ [14/10978] Consultando appid: 458590
⏳ [15/10978] Consultando appid: 302440
⏳ [16/10978] Consultando appid: 457710
⏳ [17/10978] Consultando appid: 506670
❌ Juego 506670 no disponible en Steam (success: false).
⏳ [18/10978] Consultando appid: 442080
⏳ [19/10978] Consultando appid: 222980
⏳ [20/10978] Consultando appid: 411080
⏳ [21/10978] Consultando ap

/tmp/ipykernel_31274/123985795.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combinados = pd.concat([existentes, nuevos], ignore_index=True)


⏳ [61/10978] Consultando appid: 212050
⏳ [62/10978] Consultando appid: 260510
⏳ [63/10978] Consultando appid: 418300
⏳ [64/10978] Consultando appid: 367010
⏳ [65/10978] Consultando appid: 46260
⏳ [66/10978] Consultando appid: 51100
❌ Juego 51100 no disponible en Steam (success: false).
⏳ [67/10978] Consultando appid: 397870
⏳ [68/10978] Consultando appid: 463130
⏳ [69/10978] Consultando appid: 494980
⏳ [70/10978] Consultando appid: 332150
⏳ [71/10978] Consultando appid: 299260
⏳ [72/10978] Consultando appid: 295930
⏳ [73/10978] Consultando appid: 393420
⏳ [74/10978] Consultando appid: 15210
⏳ [75/10978] Consultando appid: 327950
⏳ [76/10978] Consultando appid: 31830
⏳ [77/10978] Consultando appid: 259550
⏳ [78/10978] Consultando appid: 299340
⏳ [79/10978] Consultando appid: 328640
⏳ [80/10978] Consultando appid: 363510
⏳ [81/10978] Consultando appid: 372570
⏳ [82/10978] Consultando appid: 486470
⏳ [83/10978] Consultando appid: 452120
⏳ [84/10978] Consultando appid: 342450
⏳ [85/10978] 

/tmp/ipykernel_31274/123985795.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combinados = pd.concat([existentes, nuevos], ignore_index=True)


⏳ [91/10978] Consultando appid: 9070
⏳ [92/10978] Consultando appid: 268500
⏳ [93/10978] Consultando appid: 218660
⏳ [94/10978] Consultando appid: 452510
⏳ [95/10978] Consultando appid: 38050
⏳ [96/10978] Consultando appid: 352400
⏳ [97/10978] Consultando appid: 24670
⏳ [98/10978] Consultando appid: 217690
⏳ [99/10978] Consultando appid: 95400
⏳ [100/10978] Consultando appid: 12450
⏳ [101/10978] Consultando appid: 229580
⏳ [102/10978] Consultando appid: 292780
⏳ [103/10978] Consultando appid: 292820
⏳ [104/10978] Consultando appid: 41730
⏳ [105/10978] Consultando appid: 439900
⏳ [106/10978] Consultando appid: 25600
⏳ [107/10978] Consultando appid: 64000
⏳ [108/10978] Consultando appid: 397020
⏳ [109/10978] Consultando appid: 302670
⏳ [110/10978] Consultando appid: 349550
⏳ [111/10978] Consultando appid: 378530
⏳ [112/10978] Consultando appid: 501030
⏳ [113/10978] Consultando appid: 433950
⏳ [114/10978] Consultando appid: 255920
⏳ [115/10978] Consultando appid: 480500
⏳ [116/10978] Cons

...

⏳ [911/1292] Consultando appid: 448870
❌ Juego 448870 no disponible en Steam (success: false).
⏳ [912/1292] Consultando appid: 462030
❌ Juego 462030 no disponible en Steam (success: false).
⏳ [913/1292] Consultando appid: 333830
❌ Juego 333830 no disponible en Steam (success: false).
⏳ [914/1292] Consultando appid: 6820
❌ Juego 6820 no disponible en Steam (success: false).
⏳ [915/1292] Consultando appid: 390970
❌ Juego 390970 no disponible en Steam (success: false).
⏳ [916/1292] Consultando appid: 11170
⚠️ Error con appid 11170: 'NoneType' object has no attribute 'get'
⏳ [917/1292] Consultando appid: 100970
⚠️ Error con appid 100970: 'NoneType' object has no attribute 'get'
⏳ [918/1292] Consultando appid: 373410
⚠️ Error con appid 373410: 'NoneType' object has no attribute 'get'
⏳ [919/1292] Consultando appid: 384440
⚠️ Error con appid 384440: 'NoneType' object has no attribute 'get'
⏳ [920/1292] Consultando appid: 351040
⚠️ Error con appid 351040: 'NoneType' object has no attribute 'g

/tmp/ipykernel_31274/123985795.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combinados = pd.concat([existentes, nuevos], ignore_index=True)


⏳ [1031/1292] Consultando appid: 326740
❌ Juego 326740 no disponible en Steam (success: false).
⏳ [1032/1292] Consultando appid: 364520
❌ Juego 364520 no disponible en Steam (success: false).
⏳ [1033/1292] Consultando appid: 13700
❌ Juego 13700 no disponible en Steam (success: false).
⏳ [1034/1292] Consultando appid: 475060
❌ Juego 475060 no disponible en Steam (success: false).
⏳ [1035/1292] Consultando appid: 384110
⏳ [1036/1292] Consultando appid: 487420
❌ Juego 487420 no disponible en Steam (success: false).
⏳ [1037/1292] Consultando appid: 409370
⏳ [1038/1292] Consultando appid: 402476
❌ Juego 402476 no disponible en Steam (success: false).
⏳ [1039/1292] Consultando appid: 323430
❌ Juego 323430 no disponible en Steam (success: false).
⏳ [1040/1292] Consultando appid: 356710
⏳ [1041/1292] Consultando appid: 217490
❌ Juego 217490 no disponible en Steam (success: false).
⏳ [1042/1292] Consultando appid: 239220
❌ Juego 239220 no disponible en Steam (success: false).
⏳ [1043/1292] Cons

/tmp/ipykernel_31274/123985795.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combinados = pd.concat([existentes, nuevos], ignore_index=True)


⏳ [1171/1292] Consultando appid: 293010
❌ Juego 293010 no disponible en Steam (success: false).
⏳ [1172/1292] Consultando appid: 45740
⏳ [1173/1292] Consultando appid: 218470
❌ Juego 218470 no disponible en Steam (success: false).
⏳ [1174/1292] Consultando appid: 15270
⏳ [1175/1292] Consultando appid: 451830
❌ Juego 451830 no disponible en Steam (success: false).
⏳ [1176/1292] Consultando appid: 335590
❌ Juego 335590 no disponible en Steam (success: false).
⏳ [1177/1292] Consultando appid: 509540
⏳ [1178/1292] Consultando appid: 338540
⏳ [1179/1292] Consultando appid: 496330
❌ Juego 496330 no disponible en Steam (success: false).
⏳ [1180/1292] Consultando appid: 11120
❌ Juego 11120 no disponible en Steam (success: false).
⏳ [1181/1292] Consultando appid: 333860
❌ Juego 333860 no disponible en Steam (success: false).
⏳ [1182/1292] Consultando appid: 508960
❌ Juego 508960 no disponible en Steam (success: false).
⏳ [1183/1292] Consultando appid: 486840
❌ Juego 486840 no disponible en Stea

In [5]:
import pandas as pd
import json
import os
import requests
import time

# -------- Funciones base (reutilizadas) --------

def obtener_info_juego_completa(appid):
    url = f"https://store.steampowered.com/api/appdetails?appids={appid}"
    try:
        r = requests.get(url, timeout=10)
        data = r.json()
        juego_data = data.get(str(appid), {})
        if juego_data.get("success"):
            info = juego_data["data"]
            return {
                "appid": appid,
                "name": info.get("name"),
                "genres": [g["description"] for g in info.get("genres", [])],
                "developers": info.get("developers", []),
                "short_description": info.get("short_description", ""),
                "header_image": info.get("header_image", ""),
                "categories": [c["description"] for c in info.get("categories", [])],
                "release_date": info.get("release_date", {}).get("date", ""),
                "metacritic_score": info.get("metacritic", {}).get("score", None),
                "price": info.get("price_overview", {}).get("final_formatted", None),
            }
        else:
            print(f"❌ Juego {appid} no disponible en Steam.")
            return "no_disponible"
    except Exception as e:
        print(f"⚠️ Error con appid {appid}: {e}")
    return None


def guardar_resultados(resultados, archivo):
    if os.path.exists(archivo):
        existentes = pd.read_csv(archivo)
        nuevos = pd.DataFrame(resultados)
        combinados = pd.concat([existentes, nuevos], ignore_index=True)
        combinados = combinados.drop_duplicates(subset="appid")
    else:
        combinados = pd.DataFrame(resultados)
    combinados.to_csv(archivo, index=False)

def cargar_ya_procesados(archivo):
    if os.path.exists(archivo):
        try:
            return pd.read_csv(archivo)["appid"].astype(str).tolist()
        except:
            return []
    return []

def cargar_pendientes_json(archivo="pendientes_steam.json"):
    if os.path.exists(archivo):
        with open(archivo) as f:
            return json.load(f)
    return []

def guardar_pendientes_json(pendientes, archivo="pendientes_steam.json"):
    with open(archivo, "w") as f:
        json.dump(sorted(list(set(pendientes))), f)

# -------- Bucle principal de depuración --------

def limpiar_y_resolver_pendientes(archivo_csv="steam_juegos_metadata.csv", archivo_pendientes="pendientes_steam.json"):
    procesados = set(cargar_ya_procesados(archivo_csv))
    pendientes = set(cargar_pendientes_json(archivo_pendientes))

    while pendientes:
        print(f"🔁 Quedan {len(pendientes)} juegos pendientes")
        nuevos_pendientes = set()

        for appid in list(pendientes):
            print(f"⏳ Revisando appid: {appid}")

            if appid in procesados:
                print(f"✅ Ya está en el CSV. Eliminando de pendientes.")
                continue

            resultado = obtener_info_juego_completa(appid)

            if resultado == "no_disponible":
                print(f"🗑️ Eliminando {appid} (no disponible).")
                continue

            elif resultado is None:
                print(f"⚠️ Error técnico. Manteniendo {appid} en pendientes.")
                nuevos_pendientes.add(appid)

            else:
                print(f"💾 Guardando info de {appid}")
                guardar_resultados([resultado], archivo_csv)
                procesados.add(appid)

            time.sleep(1.1)

        pendientes = nuevos_pendientes
        guardar_pendientes_json(pendientes, archivo_pendientes)

    print("🎉 Todos los juegos pendientes han sido resueltos o descartados.")


In [6]:
limpiar_y_resolver_pendientes()

🔁 Quedan 1887 juegos pendientes
⏳ Revisando appid: 292640
❌ Juego 292640 no disponible en Steam.
🗑️ Eliminando 292640 (no disponible).
⏳ Revisando appid: 224220
❌ Juego 224220 no disponible en Steam.
🗑️ Eliminando 224220 (no disponible).
⏳ Revisando appid: 10630
❌ Juego 10630 no disponible en Steam.
🗑️ Eliminando 10630 (no disponible).
⏳ Revisando appid: 35470
❌ Juego 35470 no disponible en Steam.
🗑️ Eliminando 35470 (no disponible).
⏳ Revisando appid: 492830
❌ Juego 492830 no disponible en Steam.
🗑️ Eliminando 492830 (no disponible).
⏳ Revisando appid: 316390
❌ Juego 316390 no disponible en Steam.
🗑️ Eliminando 316390 (no disponible).
⏳ Revisando appid: 506670
❌ Juego 506670 no disponible en Steam.
🗑️ Eliminando 506670 (no disponible).
⏳ Revisando appid: 316950
❌ Juego 316950 no disponible en Steam.
🗑️ Eliminando 316950 (no disponible).
⏳ Revisando appid: 464830
❌ Juego 464830 no disponible en Steam.
🗑️ Eliminando 464830 (no disponible).
⏳ Revisando appid: 467560
❌ Juego 467560 no dis

In [ ]:
import pandas as pd
import json

# Cargar metadatos
df = pd.read_csv("steam_juegos_metadata.csv").fillna("")
df["appid"] = df["appid"].astype(str)

# Añadir columna de recomendaciones si no existe
if "recommendations" not in df.columns:
    df["recommendations"] = 0

# Si tienes un JSON o necesitas añadirlas manualmente, este es el punto:
# Por ahora asumimos que ya están integradas o a 0 si no existían

# Normalizar popularidad (basada en recomendaciones)
df["popularity_score"] = df["recommendations"].fillna(0).astype(int)
if df["popularity_score"].max() > 0:
    df["popularity_normalized"] = df["popularity_score"] / df["popularity_score"].max()
else:
    df["popularity_normalized"] = 0

# Guardar metadatos actualizados
df.to_csv("steam_juegos_metadata.csv", index=False)

# Crear diccionario para popularidad
popularidad = dict(zip(df["appid"], df["popularity_normalized"]))

# Guardar como JSON
with open("popularidad.json", "w") as f:
    json.dump(popularidad, f)

print(f"✅ Columna 'recommendations' añadida y 'popularidad.json' generado con {len(popularidad)} juegos.")
